In [ ]:
class dummySessionState:
    '''
    This is a simple class meant to mimic the SessionState class 
    from the streamlit library. It is used to store the state of the 
    app and its variables.
    '''
    def __init__(self):
        pass

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def __getitem__(self, key):
        return getattr(self, key)

In [ ]:
import subprocess
import numpy as np
import platform_io

# Import relevant libraries
import nidap_dashboard_lib as ndl   # Useful functions for dashboards connected to NIDAP
import basic_phenotyper_lib as bpl  # Useful functions for phenotyping collections of cells

def top_of_page_reqs(session_state):
    '''
    Top of the page requirements. These are the commands
    that are run at the top of the page that help maintain 
    '''

    # Initalize session_state values for streamlit processing
    session_state = ndl.init_session_state(session_state)

    # Check the platform
    session_state = check_for_platform(session_state)

    return session_state

def platform_is_nidap():
    '''
    Check if the Streamlit application is operating on NIDAP
    '''
    return np.any(['nidap.nih.gov' in x for x in subprocess.run('conda config --show channels', shell=True, capture_output=True).stdout.decode().split('\n')[1:-1]])

def check_for_platform(session_state):
    '''
    Set the platform parameters based on the platform the Streamlit app is running on
    '''
    # Initialize the platform object

    session_state['platform'] = platform_io.Platform(platform=('nidap' if platform_is_nidap() else 'local'))
    return session_state

def apply_umap(session_state, umap_style):
    '''
    Call back function for applying the UMAP functions
    '''
    clust_minmax = [1, 40]
    session_state.bc.startTimer()
    session_state.spatial_umap = bpl.perform_spatialUMAP(session_state.spatial_umap,
                                                                umap_style)

    # Record time elapsed
    session_state.bc.printElapsedTime(msg = 'Performing UMAP')
    session_state.bc.set_value_df('time_to_run_UMAP', session_state.bc.elapsedTime())

    # List of possible UMAP Lineages as defined by the completed UMAP
    session_state.umapPheno = [session_state.defLineageOpt]
    session_state.umapPheno.extend(session_state.pheno_summ['phenotype'])
    session_state.umapMarks = [session_state.defLineageOpt]
    session_state.umapMarks.extend(session_state.spatial_umap.markers)
    session_state.umapMarks.extend(['Other'])

    # Identify all of the features in the dataframe
    session_state.outcomes = session_state.spatial_umap.cells.columns

    # List of possible outcome variables as defined by the config yaml files
    session_state.umapOutcomes = [session_state.defumapOutcomes]
    session_state.umapOutcomes.extend(session_state.outcomes)
    session_state.inciOutcomes = [session_state.definciOutcomes]
    session_state.inciOutcomes.extend(session_state.outcomes)

    session_state.df_umap = session_state.spatial_umap.cells.loc[session_state.spatial_umap.cells['umap_test'], :]

    # Perform possible cluster variations with the completed UMAP
    session_state.bc.startTimer()
    session_state.clust_range, session_state.wcss = bpl.measure_possible_clust(session_state.spatial_umap, clust_minmax)
    session_state.bc.printElapsedTime(msg = 'Calculating possible clusters')

    session_state.wcss_calc_completed = True
    session_state.umapCompleted = True

    session_state = filter_and_plot(session_state)

    return session_state

def set_clusters(session_state):
    '''
    Callback function for setting the number of clusters
    and applying them to the UMAP/dataset
    '''
    session_state.bc.startTimer()
    session_state.spatial_umap = bpl.perform_clusteringUMAP(session_state.spatial_umap,
                                                            session_state.slider_clus_val)
    session_state.selected_nClus = session_state.slider_clus_val

    # Record time elapsed
    session_state.bc.printElapsedTime(msg = 'Setting Clusters')
    session_state.bc.set_value_df('time_to_run_cluster', session_state.bc.elapsedTime())

    session_state.clustering_completed = True

    session_state = filter_and_plot(session_state)

    return session_state

def filter_and_plot(session_state):
    '''
    function to update the filtering and the figure plotting
    '''
    if session_state.umapCompleted:
        session_state.df_umap_filt = session_state.spatial_umap.cells.loc[session_state.spatial_umap.cells['Slide ID'] == session_state['selSlide ID'], :]
        session_state = ndl.setFigureObjs_UMAP(session_state)

    return session_state

In [ ]:
# Import relevant libraries
import nidap_dashboard_lib as ndl   # Useful functions for dashboards connected to NIDAP
import basic_phenotyper_lib as bpl  # Useful functions for phenotyping collections of cells
import dataset_formats

session_state = dummySessionState()

# Run Top of Page (TOP) functions
session_state = top_of_page_reqs(session_state)

selectProj_u = 'C:/DATA/neighborhood-profiles/'
filename = 'input/pt_pt_pt_TnT 48h Untreated zScore areaNorm 48 h AA1 Reg_microns.csv'
# filename = 'input/ComboUntreatedNoH.csv'
# filename = 'input/Combo_CSVfiles_20230327_152849.csv'
dataset_path = selectProj_u + filename

_, _, _, _, file_format, _ = dataset_formats.extract_datafile_metadata(filename)

dataset_class = getattr(dataset_formats, file_format)
dataset_obj = dataset_class(filename, 
                            coord_units_in_microns = 1, 
                            extra_cols_to_keep=['tNt', 'GOODNUC', 'HYPOXIC', 'NORMOXIC', 'NucArea', 'RelOrientation'])
dataset_obj.process_dataset()

# Load dataset into Memory
df_load = ndl.load_dataset(session_state.fiol, filename, files_dict = 'None', file_path = 'None', loadCompass=False)

# Perform pre-processing Steps
session_state = ndl.loadDataButton(session_state, dataset_obj.data, 'Local Upload', filename[:-4])

# Update 
session_state.marker_names = ['VIM', 'ECAD', 'NOS2', 'COX2']
# session_state.marker_names = ['CD8', 'COX2', 'NOS2']
session_state = ndl.set_phenotyping_elements(session_state, session_state.df_raw)

# Apply Phenotyping Method
session_state.selected_phenoMeth = 'Species'
session_state = ndl.updatePhenotyping(session_state)

In [ ]:
# Reset the settings required for Neighborhood Analysis
session_state = ndl.reset_neigh_profile_settings(session_state)

session_state.bc.startTimer()
session_state.spatial_umap = bpl.setup_Spatial_UMAP(session_state.df,
                                                    session_state.marker_multi_sel,
                                                    session_state.phenoOrder)

session_state.spatial_umap = bpl.perform_density_calc(session_state.spatial_umap,
                                                      session_state.bc,
                                                      session_state.cpu_pool_size)

# Record time elapsed
session_state.bc.set_value_df('time_to_run_counts', session_state.bc.elapsedTime())

session_state.cell_counts_completed = True

In [ ]:
counts = session_state.spatial_umap.calculate_density_matrix_for_all_images()

In [ ]:
def check_same_array(arr1, arr2):
    if((arr1 == arr2).all() == True):
        print(" Yes!! The given arrays are equal. ")
    else:
        print(" The given arrays are not equal. ")

In [ ]:
arr1 = session_state.spatial_umap.counts
arr2 = counts

arr2

In [ ]:
df = session_state.spatial_umap.cells
df.to_csv('sample_ptptpt.csv', index=False)

In [ ]:
import multiprocessing as mp
from functools import partial

chunk = range(20)
idx = range(len(session_state.spatial_umap.cell_positions))
args = dict(cell_positions=session_state.spatial_umap.cell_positions[idx],
            cell_labels=session_state.spatial_umap.cell_labels.values[idx],
            dist_bin_px=session_state.spatial_umap.dist_bin_px)
pool_map_fn = partial(session_state.spatial_umap.process_cell_counts, **args)

results = list(map(pool_map_fn, chunk))
counts = list(map(lambda x: np.stack(x, axis=0), results))
session_state.spatial_umap.counts[chunk] = counts


In [ ]:
import dill
ptptpt_baras_counts = session_state.spatial_umap.counts
with open("density_calculation_benchmarking", "wb") as dill_file:
    dill.dump(ptptpt_baras_counts, dill_file)


In [ ]:
session_state = apply_umap(session_state, umap_style = 'Densities')

In [ ]:
session_state.slider_clus_val = 5
elbowFig = bpl.draw_wcss_elbow_plot(session_state.clust_range, session_state.wcss, session_state.slider_clus_val)

# Perform Clustering
session_state = set_clusters(session_state)

In [ ]:
df_sample = session_state.spatial_umap.cells.loc[0:200, ['Slide ID', 'Cell X Position', 'Cell Y Position', 'Lineage']]
df_sample.to_csv('sample.csv', index=False)

## Plotting

In [ ]:
session_state = ndl.setFigureObjs(session_state)

In [ ]:
NeiProFig = bpl.neighProfileDraw(session_state.spatial_umap, 3)

In [ ]:
session_state = ndl.setFigureObjs_UMAP(session_state)

In [ ]:
session_state.UMAPFigType = 'Density'
session_state.lineageDisplayToggle == 'Phenotypes'

session_state.umapInspect_Ver = 'All Phenotypes'
session_state.umapInspect_Feat = 'NucArea'
session_state.diffUMAPSel_Ver = 'All Phenotypes'
session_state.diffUMAPSel_Feat = 'NucArea'

session_state.inciOutcomeSel = 'HYPOXIC'
session_state.inciPhenoSel = 'ECAD+'
session_state = ndl.setFigureObjs_UMAPDifferences(session_state)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

dfUMAP = pd.DataFrame(data = session_state.spatial_umap.umap_test, columns = ['X', 'Y'])
dfUMAP['Cluster'] = session_state.spatial_umap.cells['clust_label'].values[session_state.spatial_umap.cells['umap_test']]
dfUMAP['Lineage'] = session_state.spatial_umap.cells['Lineage'].values[session_state.spatial_umap.cells['umap_test']]
for outcome in session_state.outcomes:
    dfUMAP[outcome] = session_state.spatial_umap.cells[outcome].values[session_state.spatial_umap.cells['umap_test']]
clustOrder = sorted(dfUMAP['Cluster'].unique())

n_bins = 200
xx = np.linspace(np.min(dfUMAP['X']), np.max(dfUMAP['X']), n_bins + 1)
yy = np.linspace(np.min(dfUMAP['Y']), np.max(dfUMAP['Y']), n_bins + 1)
n_pad = 30

minXY = dfUMAP[['X', 'Y']].min()-1
maxXY = dfUMAP[['X', 'Y']].max()+1

dfUMAPI = dfUMAP.copy()
dfUMAPD = dfUMAP.copy()

# Lineage filtering
dfUMAPI = ndl.filterLineage4UMAP(session_state, dfUMAPI, session_state.defLineageOpt, session_state.umapInspect_Ver)
dfUMAPD = ndl.filterLineage4UMAP(session_state, dfUMAPD, session_state.defLineageOpt, session_state.diffUMAPSel_Ver)

# weights for inspection umap
if session_state.umapInspect_Feat != session_state.defumapOutcomes:
    w_Ins = dfUMAPI[session_state.umapInspect_Feat]
    w_Ins, dfUMAPI = bpl.preprocess_weighted_umap(w_Ins, dfUMAPI)
else:
    w_Ins = None

# weights for difference umap
if session_state.diffUMAPSel_Feat != session_state.defumapOutcomes:
    w = dfUMAPD[session_state.diffUMAPSel_Feat]
    w, dfUMAPD = bpl.preprocess_weighted_umap(w, dfUMAPD)
    w_DiffA = w
    w_DiffB = max(w) - w
    w_Diff  = w_DiffA - w_DiffB
else:
    w_DiffA = None
    w_DiffB = None
    w_Diff  = None


In [ ]:
from scipy import ndimage as ndi
gaussian_sigma=0.5
bins=200
w = w_Diff

X = dfUMAPD['X']
Y = dfUMAPD['Y']

b, _, _ = np.histogram2d(X, Y, bins=bins)
b = ndi.gaussian_filter(b.T, sigma=gaussian_sigma)

s, _, _ = np.histogram2d(X, Y, bins=bins, weights=w)
s = ndi.gaussian_filter(s.T, sigma=gaussian_sigma)

d = np.zeros_like(b)
# d[b > 0] = s[b > 0] / b[b > 0]
d = s
d = ndi.gaussian_filter(d, sigma=gaussian_sigma)

print(np.quantile(d[d > 0].flatten(), 0.97))
print(np.quantile(d[d < 0].flatten(), 0.03))

In [ ]:
if session_state.diffUMAPSel_Feat in session_state.outcomes_nBOOL:
    compThresh = 0
    w_DiffA = np.array(w > compThresh).astype('int')
    w_DiffB = max(w) - w
else:
    w_DiffA = w
    w_DiffB = max(w) - w

print(w_DiffA)

# Import Libraries
import sys
sys.path.append('C:\GitHub\SpatialUMAP')

# import plottingTools from Baras codebase
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import PlottingTools as umPT

# load spatial_umap object
data_dir = r'C:\DATA\neighborhood-profiles'
spatial_umap = pickle.load(open(data_dir + '/pkl/spatial_umap.pkl', 'rb'))

In [ ]:
# Define a color palette for processes going forward\n",
colorPalette = np.array([[131, 201, 255], [125, 239, 161], [109, 63, 192], [255, 171, 171],\
                         [0, 104, 201], [41, 176, 157], [213, 218, 229], [255, 43, 43],\
                         [232, 197, 77], [255, 209, 106], [131, 201, 255], [160, 120, 148], \
                         [178, 70, 181], [255, 135, 0], [0, 104, 0], [82, 166, 56],\
                         [166, 63, 152], [141, 166, 42], [130, 86, 133], [133, 54, 23],\
                         [9, 89, 133], [240, 135, 228], [240, 188, 176], [113, 32, 240],\
                         [57, 240, 223], [95, 166, 94], [94, 94, 65], [94, 51, 51],\
                         [50, 94, 32], [252, 226, 17]])/256\
# umPT.draw_cmp_swatches(colorPalette)"

In [ ]:
# Define some labels for our phenotypes
phenoLabel = np.sort(spatial_umap.cells['Lineage'].unique())[::-1]
phenoColor = colorPalette

# Create a dictionary of phenotypes and the colors to draw them as
phenoSet = dict([(phenoLabel[x], phenoColor[x]) for x in range(len(phenoLabel))])

In [ ]:
import matplotlib as mpl
phenoLabel = np.sort(spatial_umap.cells['Lineage'].unique())[::-1]
phenoColor = mpl.colormaps['tab20'].colors

# Create a dictionary of phenotypes and the colors to draw them as
spatial_umap.pheno_palette_dict = dict([(phenoLabel[x], phenoColor[x]) for x in range(len(phenoLabel))])

In [ ]:
import pandas as pd
pd.__version__

### Cellular Spatial Positions
One of the first steps that will be useful is to have a view of the full cellular plate. Phenotypes can eventually be added, but for now, lets get a bird eye's view of the sample to confirm the shape of the cells.

In [ ]:
SpatPosFig = plt.figure(figsize = (7,7))
ax = SpatPosFig.add_subplot(1,1,1)
umPT.plot_spatial_elem(ax, 
                  elems = spatial_umap.cells, 
                  title=f'Spatial Cell Positions', 
                  color= spatial_umap.cells['Lineage'].map(phenoSet))

SpatPosFig.savefig(data_dir + '/plots/SampleSpatialPos.png')

In [ ]:
# umPT.plot_spatial_interactive(elems = spatial_umap.cells, title=f'Spatial Cell Positions', feature= 'Lineage')

### Neighborhood Profiles
The core of this analysis is to show varying densities of the cell phenotypes surrounding any given cell. For this reason, we need a fairly robust method of drawing this feature repeatability and comparatively.

In [ ]:
NeiProFig = plt.figure(facecolor = SlBgC, figsize = (20, 15))
NeiProFig.suptitle('Cell Densities', fontsize=55, color = SlTC)

maxDens = spatial_umap.density.max().max() # This will be a good var to use when outliers are corrected for

# Look at four (4) representative cells to get an idea of the neighborhood profiles.
for count, j in enumerate([1, 24, 2456, 4500]):
    density = spatial_umap.density[j, :, :]

    ax = NeiProFig.add_subplot(2, 2, count+1)
    if count+1 == 3:
        legF = 1
    else:
        legF = 0

    lineage = spatial_umap.cells.loc[j, "Lineage"]
    umPT.plot_neighborhood_profile(ax, f'{j}\n {lineage}', spatial_umap.dist_bin_um, density, phenoSet, 0.1, legF)
NeiProFig.savefig(data_dir + '/plots/SampleNeighborhoodProfiles.png')

In [ ]:
NeiPro_Proportion = plt.figure(facecolor = SlBgC, figsize = (20, 15))
NeiPro_Proportion.suptitle('Cell Proportions', fontsize=55, color = SlTC)

# Look at four (4) representative cells to get an idea of the neighborhood profiles.
for count, j in enumerate([1, 24, 2456, 4500]):
    propor = spatial_umap.proportion[j, :, :]

    ax = NeiPro_Proportion.add_subplot(2, 2, count+1)
    if count+1 == 4:
        legF = 1
    else:
        legF = 0

    lineage = spatial_umap.cells.loc[j, "Lineage"]
    umPT.plot_neighborhood_profile_propor(ax, f'{j}\n {lineage}', spatial_umap.dist_bin_um, propor, phenoSet, colorPalette, legF=legF)
NeiPro_Proportion.savefig(data_dir + '/plots/SampleNeighborhoodProfiles_Proportions.png')

### Spatial UMAP
Use a 2D histogram to display the heatmap of the 2-dimenstional UMAP features

In [ ]:
# Spatial UMAP 2D Density Plots By Lineage and with PD-L1 and PD1 overlays
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

# set meshgrid / bins for 2d plots based on UMAP x, y distributions
n_bins = 200
xx = np.linspace(np.min(spatial_umap.umap_test[:, 0]), np.max(spatial_umap.umap_test[:, 0]), n_bins + 1)
yy = np.linspace(np.min(spatial_umap.umap_test[:, 1]), np.max(spatial_umap.umap_test[:, 1]), n_bins + 1)
n_pad = 30

# get figure and axes
sUMAPFig, ax = plt.subplots(4, 5, figsize=(24, 20), facecolor='white')

# color maps
cmap_dens = plt.get_cmap('viridis').copy()
cmap_dens.set_under('white')
cmap_div = plt.get_cmap('coolwarm').copy()

# plot cmaps
umPT.plt_cmap(ax=ax[1, 4], cmap=cmap_dens, extend='max', width=0.01, ylabel='Density')
umPT.plt_cmap(ax=ax[2, 4], cmap=cmap_div, extend='both', width=0.01, ylabel='Rel. Nucleus Ori')
umPT.plt_cmap(ax=ax[3, 4], cmap=cmap_div, extend='both', width=0.01, ylabel='Nucleus Area')

# clear unneeded axes
[ax[_].set(visible=False) for _ in [(0, 0), (0, 1), (0, 3), (0, 4)]]

# plot 2d denisty in umap of all cells
umPT.plot_2d_density(spatial_umap.umap_test[:, 0], spatial_umap.umap_test[:, 1], bins=[xx, yy], n_pad=n_pad, ax=ax[0, 2], cmap=cmap_dens)
ax[0, 2].set(title='All Cells')

# set lineages to show and in what order
lineages_plot = ['VIM', 'ECAD', 'COX2', 'NOS2']

# Secondary Characteristics-Rel Orientation
w = {'relOrientation': spatial_umap.cells['RelOrientation'].values}

# Secondary Characteristics-Nuc Area
w['NucArea'] = spatial_umap.cells['NucArea'].values

for i in range(len(lineages_plot)):
    # cells of lineage(s)
    idx = spatial_umap.cells['Lineage'].str.contains(f'{lineages_plot[i]}\+')
    ax[1, i].cla()
    ax[1, i].set(title=lineages_plot[i])

    # plot density
    umPT.plot_2d_density(spatial_umap.umap_test[idx, 0], spatial_umap.umap_test[idx, 1], bins=[xx, yy], ax=ax[1, i], cmap=cmap_dens, vlim=.95)

    # plot Secondary Pheno 1
    umPT.plot_2d_density(spatial_umap.umap_test[idx, 0], spatial_umap.umap_test[idx, 1], bins=[xx, yy], w=w['relOrientation'][idx], ax=ax[2, i],
                    cmap=cmap_div, vlim=np.array([spatial_umap.cells['RelOrientation'].min(), spatial_umap.cells['RelOrientation'].max()]), circle_type='arch')
    # plot Secondary Pheno 2
    umPT.plot_2d_density(spatial_umap.umap_test[idx, 0], spatial_umap.umap_test[idx, 1], bins=[xx, yy], w=w['NucArea'][idx], ax=ax[3, i], 
                    cmap=cmap_div, vlim=np.array([spatial_umap.cells['NucArea'].min(), spatial_umap.cells['NucArea'].max()]), circle_type='arch')

sUMAPFig.savefig(data_dir + '/plots/Sample-SpatialUMAP.png')

### Spatial Coord/UMAP

In [ ]:
clust_set = np.sort(spatial_umap.cells['clust_label'].unique())
color_set = dict([(clust_set[x], colorPalette[x]) for x in range(len(clust_set))])

SpatPosClustFig = plt.figure(figsize = (7,7))
ax = SpatPosClustFig.add_subplot(1,1,1)
umPT.plot_spatial_elem(ax, 
                  elems = spatial_umap.cells, 
                  title=f'Spatial Cell Positions\nWith Spatial-UMAP cluster Overlay', 
                  color=spatial_umap.cells['clust_label'].map(color_set))

SpatPosClustFig.savefig(data_dir + '/plots/SampleSpatialPosUMAP.png')

In [ ]:
# umPT.plot_spatial_interactive(elems = spatial_umap.cells, title=f'Spatial Cell Positions-UMAP Overlay', feature= 'clust_label')

### Meaned Measures (Density/Proportion)

In [ ]:
# Max density value across all densities
maxDens = spatial_umap.density.max().max() # This will be a good var to use when outliers are corrected for

# Create figure and axes
NeiProFigMean = plt.figure(figsize=(12,12), facecolor = SlBgC)
ax = NeiProFigMean.add_subplot(1, 1, 1, facecolor = SlBgC)

umPT.plot_mean_neighborhood_profile(ax, 10, spatial_umap.dist_bin_um, spatial_umap.densMeansDict, spatial_umap.pheno_palette_dict, 0.1, legF=1)